<a href="https://colab.research.google.com/github/rajaehilali/PFE/blob/main/YOLO%20V11%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install ultralytics -q
import os
from ultralytics import YOLO

os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "dryrun"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.6 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# Unzip your uploaded files into /content/data
!mkdir -p /content/data
!unzip -q "/content/drive/MyDrive/wetransfer_valid_2025-07-21_1338.zip" -d /content/data
!unzip -q "/content/drive/MyDrive/wetransfer_test_2025-07-21_1204.zip" -d /content/data


In [6]:
yaml_path = "/content/data/data.yaml"
data_yaml = """
train: /content/data/valid/images
val: /content/data/valid/images   # we use the same for val since no train set provided
test: /content/data/test/images

nc: 1
names: ["panel"]
"""

with open("/content/data/data.yaml", "w") as f:
    f.write(data_yaml)

In [7]:
print("✅ data.yaml created at:", yaml_path)
!cat /content/data/data.yaml

✅ data.yaml created at: /content/data/data.yaml

train: /content/data/valid/images
val: /content/data/valid/images   # we use the same for val since no train set provided
test: /content/data/test/images

nc: 1
names: ["panel"]


In [9]:
from ultralytics import YOLO

# Load YOLO11 model (nano version, you can change to s/m/l if GPU allows)
model = YOLO("yolo11n.pt")

# Train with early stopping
model.train(
    data="/content/data/data.yaml",   # dataset yaml
    epochs=200,                       # max, will stop earlier
    imgsz=512,
    batch=8,
    patience=20,                      # stop if no improvement after 20 epochs
    save_period=5,
    name="yolo11_panels_earlystop"
)



Ultralytics 8.3.200 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11_panels_earlystop, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0, plots=

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7dec460e4710>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [10]:
# Evaluate after training
metrics = model.val()
print("✅ Evaluation metrics:", metrics)

# Detect + crop panels for CNN
results = model.predict(
    source="/content/data/test/images",
    conf=0.25,
    save=True,
    save_txt=True,
    save_crop=True
)
print("Crops saved in /content/runs/detect/predict/crops/panel/")

Ultralytics 8.3.200 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4300.1±1485.6 MB/s, size: 739.6 KB)
val: Scanning /content/data/valid/labels.cache... 195 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 195/195 342.2Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 25/25 4.2it/s 6.0s
                   all        195        195      0.966      0.877      0.943      0.752
Speed: 0.2ms preprocess, 5.6ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to /content/runs/detect/yolo11_panels_earlystop2
✅ Evaluation metrics: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7debf833b020>
curves: ['Prec